In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Define the Network

In [46]:
class Net(nn.Module): 
    def __init__(self):
        super(Net,self).__init__()  # 다수의 클래스 상속 시 생성자가 여러번 호출되는 것을 방지하기 위해 super()사용
        self.conv1 = nn.Conv2d(1, 6, 3) # 1 input image channel, 6 output image channels, 3*3 square convolution kernel
        self.conv2 = nn.Conv2d(6, 16, 3) # 6 input image channel, 16 output image channels, 3*3 square convolution kernel
        self.fc1 = nn.Linear(16*6*6, 120) # 6*6은 image dimension을 의미한다
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # if the size is a square you can only specify a single number. 2라고 써도 (2,2)로 맥스풀링하는 것
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


+ the backward function is automatically defined for you using autograd. 
+ you can use any of the Tensor operations in the forward function

In [47]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [21]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0358, -0.0124,  0.0019,  0.0013,  0.0725,  0.0276,  0.1231, -0.1268,
          0.0720,  0.1161]], grad_fn=<AddmmBackward>)


In [22]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [23]:
print(input.grad)

None


#### Recap
+ torch.Tensor : A multi-dimensional array with support for autograd operations like backward()
+ nn.Module : Neural Network Module. convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
+ nn.Parameter : A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module. 
+ autograd.Function : Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its history

## Loss Function

In [48]:
output = net(input)
target = torch.randn(10) # a dummy target
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)

In [49]:
print(loss)

tensor(0.3809, grad_fn=<MseLossBackward>)


In [50]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## BackProp

In [51]:
net.zero_grad()   #zero the gradient buffers lest gradients should be accumulated

In [38]:
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])


In [39]:
loss.backward()

In [40]:
print(net.conv1.bias.grad)

tensor([-0.0078,  0.0026,  0.0143,  0.0156,  0.0162,  0.0048])


In [43]:
learning_rate = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [45]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad() #zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()